# AMUSE for NEMO users

Make sure you start this notebook with NEMO preloaded in your shell, else the **%%bash** cells will not easily work. 

## 1. 

First, two sanity checks.  Is NEMO loaded. Is amuse available?

In [ ]:
%%bash

nemo

## 2.

Check if **AMUSE** can be imported, and where it's located.

In [ ]:
import amuse
print(amuse.__file__)

## 3.

Get the modules from AMUSE we need

In [ ]:
from amuse.units import nbody_system
from amuse.ic.plummer import new_plummer_model
from amuse.community.bhtree import Bhtree
from amuse.io import write_set_to_file


Create a small Plummer sphere and print the contents

In [ ]:
np.random.seed(123)
stars = new_plummer_model(8)

print(stars)

In [ ]:
Set up an nbody integrator, and evolve the system

In [ ]:
gravity = Bhtree()

eps = 0.025 | nbody_system.length
tstop = 10  | nbody_system.time

gravity.parameters.opening_angle  = 0.75
gravity.parameters.epsilon_squared = eps * eps
gravity.parameters.dt_dia          = 1.0      | nbody_system.time
gravity.parameters.timestep        = 0.015625 | nbody_system.time       # 1/64

stars_in_gravity = gravity.particles.add_particles(stars)
gravity.evolve_model(tstop)



In [ ]:
# https://github.com/amusecode/amuse/issues/1121

# help(gravity.parameters)

Write the model to an ASCII table 

In [ ]:

write_set_to_file(stars_in_gravity, 'plummer.txt', format='txt')



Back to NEMO, convert the ASCII table into a NEMO format

In [ ]:
%%bash

head -3 plummer.txt

rm -f plummer.snap
tabcols plummer.txt | tabtos - plummer.snap block1=mass,skip,vx,vy,vz,x,y,z nbody=$(tabcols plummer.txt | wc -l)
tsf plummer.snap
